In [ ]:
"""Utility script for handling JLK brain CT data and generating reports."""

from __future__ import annotations

from pathlib import Path

from dataset_utils import _collection_path
from file_utils import extract_zip
from image_processing import convert_all_dicom_to_png_parallel
from openai_utils import JLK_ICH, JLK_CTL, JLK_CTI, JLK_WMHC
from tqdm import tqdm

def main(zip_file) -> None:
    """Example main routine."""

    zip_path = Path(zip_file)
    extract_path = extract_zip(zip_path)
    if not extract_path:
        return

    df = _collection_path(extract_path)
    output_dir = Path(f"{Path(zip_file).stem}_output_images")
    convert_all_dicom_to_png_parallel(df, output_dir)

    AI_result = []
    for item in tqdm(output_dir.iterdir()):
        if item.is_dir():
            non_mask_dir = item / "Non_mask"
            if non_mask_dir.exists() and any(non_mask_dir.iterdir()):
                # ICH
                ich_dir = next((p for p in item.iterdir() if p.is_dir() and "ICH" in p.name), None)
                result_ich = JLK_ICH(non_mask_dir, ich_dir)
                AI_result.append({
                    "solution": "ICH",
                    "result": result_ich,
                    "non_mask_dir": str(non_mask_dir),
                    "ai_dir": str(ich_dir) if ich_dir else None
                })

                # CTL
                ctl_dir = next((p for p in item.iterdir() if p.is_dir() and "CTL" in p.name), None)
                result_ctl = JLK_CTL(non_mask_dir, ctl_dir)
                AI_result.append({
                    "solution": "CTL",
                    "result": result_ctl,
                    "non_mask_dir": str(non_mask_dir),
                    "ai_dir": str(ctl_dir) if ctl_dir else None
                })

                # CTI
                cti_dir = next((p for p in item.iterdir() if p.is_dir() and "CTI" in p.name), None)
                result_cti = JLK_CTI(non_mask_dir, cti_dir)
                AI_result.append({
                    "solution": "CTI",
                    "result": result_cti,
                    "non_mask_dir": str(non_mask_dir),
                    "ai_dir": str(cti_dir) if cti_dir else None
                })

                # WMHC
                wmhc_dir = next((p for p in item.iterdir() if p.is_dir() and "WMHC" in p.name), None)
                result_wmhc = JLK_WMHC(non_mask_dir, wmhc_dir)
                AI_result.append({
                    "solution": "WMHC",
                    "result": result_wmhc,
                    "non_mask_dir": str(non_mask_dir),
                    "ai_dir": str(wmhc_dir) if wmhc_dir else None
                })



if __name__ == "__main__":  # pragma: no cover - manual execution
    main("/home/yjpark/llm_env/dicom_project_template/DCM_REQUEST_2025-07-23-05-16-58-465027_0.zip")


  4%|▍         | 18/445 [00:07<03:07,  2.28it/s]


In [1]:
"""Utility script for handling JLK brain CT data and generating reports."""

from __future__ import annotations

from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed

from dataset_utils import _collection_path
from file_utils import extract_zip
from image_processing import convert_all_dicom_to_png_parallel
from openai_utils import JLK_ICH, JLK_CTL, JLK_CTI, JLK_WMHC
from tqdm import tqdm


def run_jlk_solutions(non_mask_dir: Path, item: Path) -> list[dict]:
    def run(name: str, func):
        ai_dir = next((p for p in item.iterdir() if p.is_dir() and name in p.name), None)
        return {
            "solution": name,
            "result": func(non_mask_dir, ai_dir),
            "non_mask_dir": str(non_mask_dir),
            "ai_dir": str(ai_dir) if ai_dir else None
        }

    solution_funcs = [
        ("ICH", JLK_ICH),
        ("CTL", JLK_CTL),
        ("CTI", JLK_CTI),
        ("WMHC", JLK_WMHC)
    ]

    results = []
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(run, name, func) for name, func in solution_funcs]
        for f in as_completed(futures):
            try:
                results.append(f.result())
            except Exception as e:
                print(f"[ERROR] {e}")
    return results


def main(zip_file) -> None:
    """Example main routine."""

    zip_path = Path(zip_file)
    extract_path = extract_zip(zip_path)
    if not extract_path:
        return

    df = _collection_path(extract_path)
    output_dir = Path(f"{Path(zip_file).stem}_output_images")
    convert_all_dicom_to_png_parallel(df, output_dir)

    AI_result = []
    for item in tqdm(list(output_dir.iterdir())[:3]):
        if item.is_dir():
            non_mask_dir = item / "Non_mask"
            if non_mask_dir.exists() and any(non_mask_dir.iterdir()):
                AI_result.extend(run_jlk_solutions(non_mask_dir, item))

    return AI_result
if __name__ == "__main__":  # pragma: no cover - manual execution
    rr = main("/home/yjpark/llm_env/dicom_project_template/DCM_REQUEST_2025-07-23-05-16-58-465027_0.zip")


100%|██████████| 3/3 [01:13<00:00, 24.47s/it]


In [2]:
rr

[{'solution': 'WMHC',
  'result': '{\n  "patientInfo": {\n    "patientId": "Unknown",\n    "gender": "Unknown",\n    "age": -1,\n    "imagingModalities": [\n      "NCCT"\n    ],\n    "scanTimestamp": "Unknown"\n  },\n  "interpretation": {\n    "primaryDiagnosis": "No acute intracranial hemorrhage or definite early ischemic changes are identified on this non-contrast CT (NCCT) scan.",\n    "pathophysiologicalSynopsis": "This is a non-contrast CT of the brain, demonstrating multiple axial slices from skull base to vertex. There is no evidence of acute parenchymal hemorrhage, sizable territorial hypodensity, or significant mass effect. Gray-white differentiation is preserved in the basal ganglia and cortex, and there are no obvious signs of early ischemic injury (e.g., sulcal effacement, loss of insular ribbon, obscuration of lentiform nucleus). No midline shift or ventriculomegaly is seen. The scan overall shows a structurally preserved brain without acute catastrophic changes.",\n    "a